In [3]:
%pip install -q -U trl numpy peft transformers datasets bitsandbytes huggingface_hub tqdm codecarbon

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.6 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from datasets import load_dataset, Dataset
from scipy.special import softmax
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from sklearn.metrics import accuracy_score, f1_score, log_loss, confusion_matrix
from transformers import pipeline, set_seed, TrainingArguments, Trainer, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM, BitsAndBytesConfig, DataCollatorWithPadding
from huggingface_hub import notebook_login, login
import json
import pandas as pd
import os
from tqdm import tqdm
import random
from codecarbon import EmissionsTracker, track_emissions

In [10]:
# Check if GPUs are available and move model to GPUs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

#model_name = 'meta-llama/Meta-Llama-3-8B'
#model_name = 'meta-llama/Llama-2-7b-hf'
#model_name = 'meta-llama/Llama-2-13b-chat-hf'
#model_name = 'meta-llama/Llama-2-70b-hf'
#model_name = "/kaggle/input/llama-3/transformers/8b-hf/1"

#model_name = "unsloth/llama-3-70b-bnb-4bit"

#save_path = f"models/{model_name}"
#print(f"Saving model to / Loading model from:\n\n -> {save_path}")


In [12]:
#quantization configurations - so you quantize the model while inferencing
bnb_config = BitsAndBytesConfig(
    #load_in_4bit = True,
    load_in_8bit=True,
    llm_int8_skip_modules=["lm_head"]
    #llm_int8_threshold=6.0,
    #bnb_4bit_qunat_type = "nf4",
    #bnb_4bit_compute_dtype = torch.float16,
)

In [13]:
login()

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.bos_token

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_8bit=True,
    #model_path,
    #num_labels=4,  # Adjust if necessary based on the specific task
    quantization_config=bnb_config,
    device_map="auto",
    #device_map=device_map,
    #load_in_8bit_fp32_cpu_offload=True,
)
model.config.pad_token_id = tokenizer.pad_token_id

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

# Test the Model

In [8]:
# List of words that should be used to create sentences.

words = [
    "apple", "book", "car", "dog", "elephant", "forest", "guitar", "house", 
    "island", "jacket", "kangaroo", "lamp", "mountain", "notebook", "ocean", 
    "pencil", "queen", "river", "star", "tree", "umbrella", "village", 
    "window", "xylophone", "yacht", "zebra", "balloon", "camera", "desert", 
    "engine", "flower", "garden", "honey", "iceberg", "jungle", "kite", 
    "ladder", "moon", "nest", "octopus", "pirate", "quilt", "robot", "swan", 
    "telescope", "unicorn", "violin", "whale", "x-ray", "yogurt"
]

In [9]:
# Write a Text Generation Function
#@track_emissions(project_name=model_name)
def generate_text(prompt, max_length=25):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids, 
            #max_length=max_length,
            pad_token_id=tokenizer.eos_token_id,
            max_new_tokens=max_length
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [12]:
def create_prompt(word): 
    
    prompt = f"""
        You are an AI-Assistant that tells jokes for a given word. \n
        You only answer with a 2 sentences long joke and you do not follow up with questions of any form.\n
        Tell me one short joke about "{word}".\n\n
        """
    
    return prompt

In [13]:
tracker = EmissionsTracker(project_name=model_name, log_level="error", pue = 1.22, output_file=f'{model_name}_emissions.csv')
tracker.start()

# Number of runs
n = 1

# Run the function for each word in the array
for i in range(n):
    print("="*25 + f" Run {i+1} " + "="*25)
    print("\n")
    for idx, word in enumerate(words):
        print("-"*25 + f" Word {idx+1} " + "-"*25)
        prompt = create_prompt(word)
        output = generate_text(prompt)
        print(output)
        print("\n")
    print("\n")
    
emissions: float = tracker.stop()
print(emissions)

========================= Run 1 =========================


------------------------- Word 1 -------------------------

        You are an AI-Assistant that tells jokes for a given word. 

        You only answer with a 2 sentences long joke and you do not follow up with questions of any form.

        Tell me one short joke about "apple".


         Here is the joke:

Why did the apple join the gym? Because it wanted to get some core strength! It's a bit corny, but apples don't get many workout opportunities!


------------------------- Word 2 -------------------------

        You are an AI-Assistant that tells jokes for a given word. 

        You only answer with a 2 sentences long joke and you do not follow up with questions of any form.

        Tell me one short joke about "book".


         Here is your answer:

        Why did the book go to therapy? Because it had a bad chapter in its life! And now it's trying to turn the page!


------------------------- Word 3 -------------

/opt/conda/lib/python3.10/site-packages/codecarbon/output_methods/file.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame.from_records([dict(data.values)])])
